In [1]:
import pandas as pd
import py2neo
import urllib.request as urllib
from py2neo import Graph,Node,Relationship


In [2]:
### esc =  eurovision song contest
#https://www.kaggle.com/datasets/datagraver/eurovision-song-contest-scores-19752019?resource=download
esc_1975_2019 =  "eurovision_song_contest_1975_2019.xlsx"

esc_additional = "Eurovision additional.xlsx"

#http://www.cepii.fr/cepii
geo_cepii = "geo_cepii.xls"

#https://github.com/geodatasource/country-borders/blob/master/GEODATASOURCE-COUNTRY-BORDERS.CSV
geo_borders = "GEODATASOURCE-COUNTRY-BORDERS.csv"

#https://datahub.io/core/country-list#resource-data
iso2 = "iso2.csv"

#load data in pandas df
df_iso2_convert = pd.read_csv("data/" + iso2)
df_borders = pd.read_csv("data/" + geo_borders)
df_geo = pd.read_excel("data/" + geo_cepii,header=0)
df_additional = pd.read_excel("data/" + esc_additional, header=0)

df_esc = pd.read_excel("data/" + esc_1975_2019,sheet_name = "Data",header=0)
df_esc.drop(["Duplicate" , "Edition"], axis =1)
df_features = []
faulty_countries = []
df_country_lookup = []

In [7]:

def get_faulty_countries(country):
    global faulty_countries
    code = df_iso2_convert.loc[df_iso2_convert['Name'] == country]["Code"].values
    if code.size == 0:
        if country not in faulty_countries:
            faulty_countries.append(country)

def fix_country(country):
    global df_country_lookup
    new_country = df_country_lookup.loc[df_country_lookup["faulty"] == country]["correct"].values 
    if new_country.size > 0:
        return new_country[0]
    else: 
        return "Nan"
    
def get_iso2(country):
    code = df_iso2_convert.loc[df_iso2_convert['Name'] == country]["Code"].values
    if code.size > 0:
        return code[0]
    else:
        return df_iso2_convert.loc[df_iso2_convert['Name'] == fix_country(country)]["Code"].values[0]

def country_to_iso2(df, columns = [] , lookup_table = []): # match the order of lookup_table to faulty countries
    global faulty_countries
    global df_country_lookup
    faulty_countries = []
    for column in columns:
        df[column].apply(get_faulty_countries)
        try: 
            df_country_lookup = pd.DataFrame({'faulty': faulty_countries, 'correct': lookup_table})
        except:
            print("lookup table does not size of faulty_countries table : \n" + "lookup: " + str(lookup_table) + "\n faulty: " + str(faulty_countries) )
            break
        newname = column + '_iso2'
        df[newname] = df[column].apply(get_iso2)
        #df.drop([column], axis=1)
        

In [8]:

lookup = ['Netherlands','Macedonia, the Former Yugoslav Republic of', 'Bosnia and Herzegovina', 'Russian Federation', 'Macedonia, the Former Yugoslav Republic of','Serbia','Moldova, Republic of', 'Macedonia, the Former Yugoslav Republic of','Netherlands', 'Macedonia, the Former Yugoslav Republic of']
country_to_iso2(df_esc, ['From country', 'To country'], lookup )
        
lookup = ['Macedonia, the Former Yugoslav Republic of', 'Moldova, Republic of', 'Russian Federation']
country_to_iso2(df_additional, ["Unnamed: 0"], lookup)

In [11]:
### create a lookup table for iso2
iso2_list = []
for index, row in df_esc.iterrows():   
    iso2 = row["From country_iso2"]
    if iso2 not in iso2_list:
        iso2_list.append(iso2)
df_iso2 = df_iso2_convert[df_iso2_convert['Code'].isin(iso2_list)].set_index('Code')
df_features = df_iso2.copy()

In [12]:
def copy_features(df,iso2_column, features = []):    
    for feature in features:
        df_features[feature] = ""
        for index, row in df_features.iterrows():
            val = df.loc[df[iso2_column]== index][feature].values
            if val.size > 0:
                df_features.loc[index,feature] = val[0]
            else:
                row[feature] = "Nan"

In [16]:
geo_features = ['continent', 'langoff_1','lat','lon','colonizer1'] #features that seem interesting? 
copy_features(df_geo,"iso2", geo_features)

additional_features = ['Religion', 'Most Common Country of Origin of Immigrants'] #features that seem interesting? 
copy_features(df_additional,"Unnamed: 0_iso2",additional_features)

In [30]:

### copy features from df_geo into the iso2 lookup table



for feature in geo_features:
    df_features[feature] = ""
    for index, row in df_features.iterrows():
        val = df_geo.loc[df_geo["iso2"]== index][feature].values
        if val.size > 0:
            df_features.loc[index,feature] = val[0]
        else:
            row[feature] = "Nan"

### copy features from df_additional into the iso2 lookup table


for feature in additional_features:
    df_features[feature] = ""
    for index, row in df_features.iterrows():
        val = df_additional.loc[df_geo["iso2"]== index][feature].values
        if val.size > 0:
            df_features.loc[index,feature] = val[0]
        else:
            row[feature] = "Nan"

,Year,(semi-) final,Jury or Televoting,Points,from_iso2,to_iso2
0,1975,f,J,0,BE,BE
1,1975,f,J,0,BE,FI
2,1975,f,J,2,BE,FR
3,1975,f,J,0,BE,DE
4,1975,f,J,12,BE,IE
...,...,...,...,...,...,...
49827,2019,f,J,0,BY,GR
49828,2019,f,J,8,BY,DE
49829,2019,f,J,0,BY,BY
49830,2019,f,J,0,BY,NL


In [150]:
df_esc

,Year,(semi-) final,Edition,Jury or Televoting,From country,To country,Points,Duplicate,from_iso2,to_iso2,continent,langoff_1,lat,lon,colonizer1
0,1975,f,1975f,J,Belgium,Belgium,0,x,BE,BE,Europe,Dutch,50.83333,4.35,.
1,1975,f,1975f,J,Belgium,Finland,0,NaN,BE,FI,Europe,Dutch,50.83333,4.35,.
2,1975,f,1975f,J,Belgium,France,2,NaN,BE,FR,Europe,Dutch,50.83333,4.35,.
3,1975,f,1975f,J,Belgium,Germany,0,NaN,BE,DE,Europe,Dutch,50.83333,4.35,.
4,1975,f,1975f,J,Belgium,Ireland,12,NaN,BE,IE,Europe,Dutch,50.83333,4.35,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49827,2019,f,2019f,J,Belarus,Greece,0,NaN,BY,GR,Europe,Belarussian,53.85,27.5,RUS
49828,2019,f,2019f,J,Belarus,Germany,8,NaN,BY,DE,Europe,Belarussian,53.85,27.5,RUS
49829,2019,f,2019f,J,Belarus,Belarus,0,x,BY,BY,Europe,Belarussian,53.85,27.5,RUS
49830,2019,f,2019f,J,Belarus,The Netherlands,0,NaN,BY,NL,Europe,Belarussian,53.85,27.5,RUS


In [6]:
# Connect to neo4j database
f=open("authentication/neo4j.txt","r")
lines=f.readlines()
user=lines[0].replace('\n','')
pwd=lines[1].replace('\n','')
http_port = lines[2]
f.close()

graph = Graph(http_port, auth=(user, pwd))

In [7]:
# laod data in neo4j database
# //Eurovision votes in the 2012,2013, 2014 and 2015 final session. (data already included in esc_1975_2019 data set? )
fp = urllib.urlopen("https://goz39a.s3.eu-central-1.amazonaws.com/eurovision/eurovision.txt")
mybytes = fp.read()
data = mybytes.decode("utf8")
fp.close()
graph.run(data)
graph.run("MATCH (n) RETURN count(*)")

count(*)
46


In [6]:

def clear_graph(): 
    return (graph.run("MATCH (n)-[r]->() DELETE r"), graph.run("MATCH (n) DELETE n"))
    
clear_graph()

((No data), (No data))